# Extreme Gradient Boost


### Initialization

In [ ]:

from xgboost import XGBRegressor
import xgboost as xgb
from utility import start
from utility import train
from utility import display

X_train, X_test, y_train, y_test, df = start("encoded_data_04_18.csv")
df.shape

In [ ]:
#model = XGBRegressor(n_estimators=2000, max_depth=10, random_state=42, learning_rate=0.25, tree_method="hist")
#odel = xgb()
# Defining hyperparameters for XGBoost
params = {
    'objective': 'reg:absoluteerror',  # for regression
    'max_depth': 7,                    # maximum depth of a tree
    'learning_rate': 0.25,              # learning rate
    'n_estimators': 5000,              # number of trees
    'lamda': 0.9,                      # regularization parameter
    'subsample': 0.8,                  # subsample ratio of the training instances
    'colsample_bytree': 0.8,           # subsample ratio of columns when constructing each tree
    'min_child_weight': 1,             # minimum sum of instance weight needed in a child
}

# Instantiating the XGBoost regressor
model = xgb.XGBRegressor(**params)

y_pred, y_pred_original, y_test_original, y_train_pred_original, _, _ = train(model, X_train, X_test, y_train, y_test)
#1.51, 41% - 1.76, 49%

Training Set Scores:
Mean Absolute Error (MAE): 0.15 M
Mean Absolute Percentage Error (MAPE): 8.64 %

Test Set Scores:
Mean Absolute Error (MAE): 1.0 M
Mean Absolute Percentage Error (MAPE): 57.87 %

In [ ]:
import graphviz
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, export_graphviz

import shap


# Creating a SHAP explainer object
explainer = shap.TreeExplainer(model)

# Calculating SHAP values for the testing data
shap_values = explainer.shap_values(X_test)

# Visualizing the SHAP summary plot
shap.summary_plot(shap_values, X_test)

In [ ]:
# Selecting a random subset of samples for visualization
sample_indices = np.random.choice(X_test.shape[0], 10, replace=False)
X_test_subset = X_test.iloc[sample_indices]

# Calculating SHAP values for the subset of testing data
shap_values = explainer.shap_values(X_test_subset)

# Visualizing the individual decision trees with SHAP
shap.decision_plot(explainer.expected_value, shap_values, X_test_subset)

In [ ]:
shap_values

In [ ]:
# Creating a SHAP explainer object
explainer = shap.TreeExplainer(model)

# Calculating SHAP values for the testing data
shap_values = explainer.shap_values(X_test)

# Calculating SHAP summary values
shap_summary_values = np.abs(shap_values).mean(axis=0)

# Identifying the indices of the top 10 most valuable players
top_10_indices = shap_values.argsort()[-10:][::-1]
print(top_10_indices)

# Selecting the top 10 most valuable players from the testing data
X_top_10 = X_test.iloc[top_10_indices]

# Calculating SHAP values for the top 10 most valuable players
shap_values_top_10 = explainer.shap_values(X_top_10)

# Visualizing the individual decision trees with SHAP for the top 10 most valuable players
shap.decision_plot(explainer.expected_value, shap_values_top_10, X_top_10)

In [ ]:
results, z = display(df, X_test, y_pred)
results


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
# Calculate distances of each point from the optimal line
distances = np.abs(y_pred_original - y_test_original)

# Find the indices of points with maximum distances
max_distance_indices = np.argsort(distances)[-30:]  # Adjust the number to show more or fewer names

# Get the corresponding player names
furthest_names = z.iloc[max_distance_indices]['name'].tolist()

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test_original, y_pred_original, s=20, color='blue', alpha=0.5, marker='o', label='Predicted Values')
plt.xlabel("Real Value (EUR)")
plt.ylabel("Predicted Value (EUR)")
plt.title("Real Value vs Predicted Value")
plt.plot([min(y_test_original), max(y_test_original)], [min(y_test_original), max(y_test_original)], color='red', label='Perfect Prediction Line')
plt.text(min(y_test_original), max(y_pred_original), "Overestimation", fontsize=10, color='green', verticalalignment='top', horizontalalignment='left')
plt.text(max(y_test_original), min(y_pred_original), "Underestimation", fontsize=10, color='orange', verticalalignment='bottom', horizontalalignment='right')

# Annotate only the furthest points
for index, row in z.iterrows():
    if row['name'] in furthest_names:
        plt.annotate(row['name'], (row['market_value_in_eur'], row['Predicted_Value']), fontsize=6, color='black')

# Set formatter for x-axis and y-axis
def millions_formatter(x, pos):
    return f'{x / 1000000:.0f}M'

formatter = FuncFormatter(millions_formatter)
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.grid(alpha=0.2, linestyle='--', zorder=1)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
y_train.shape
y_train_original = np.power(10, y_train)
y_train_original.shape
print(y_train_original.shape, y_train_pred_original.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
# Calculate distances of each point from the optimal line

y_train_original = np.power(10, y_train)
distances = np.abs(y_train_pred_original - y_train_original)

# Find the indices of points with maximum distances
max_distance_indices = np.argsort(distances)[-10:]  # Adjust the number to show more or fewer names

# Get the corresponding player names
#furthest_names = z.iloc[max_distance_indices]['name'].tolist()

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_train_original, y_train_pred_original, s=20, color='blue', alpha=0.5, marker='o', label='Predicted Values')
plt.xlabel("Real Value (EUR)")
plt.ylabel("Predicted Value (EUR)")
plt.title("Real Value vs Predicted Value")
plt.plot([min(y_train_pred_original), max(y_train_pred_original)], [min(y_train_pred_original), max(y_train_pred_original)], color='red', label='Perfect Prediction Line')
plt.text(min(y_train_original), max(y_train_pred_original), "Overestimation", fontsize=10, color='green', verticalalignment='top', horizontalalignment='left')
plt.text(max(y_train_original), min(y_train_pred_original), "Underestimation", fontsize=10, color='orange', verticalalignment='bottom', horizontalalignment='right')

# Annotate only the furthest points
#for index, row in z.iterrows():
#    if row['name'] in furthest_names:
#        plt.annotate(row['name'], (row['market_value_in_eur'], row['Predicted_Value']), fontsize=6, color='black')

# Set formatter for x-axis and y-axis
def millions_formatter(x, pos):
    return f'{x / 1000000:.0f}M'

formatter = FuncFormatter(millions_formatter)
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.grid(alpha=0.2, linestyle='--', zorder=1)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Scatter plot
plt.scatter(y_test, y_pred, s=20, color='blue', alpha=0.5)  # Set dot size to 20, blue color, and transparency to 0.5
plt.xlabel("Real value")
plt.ylabel("Predicted value")
plt.title("Value vs Predicted value")

# Add diagonal line
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.text(min(y_test), max(y_pred), "Over evaluation", fontsize=10, color='green', verticalalignment='top', horizontalalignment='left')
plt.text(max(y_test), min(y_pred), "Under evaluation", fontsize=10, color='orange', verticalalignment='bottom', horizontalalignment='right')
plt.grid(alpha=0.2, zorder=1)

plt.show()


In [ ]:
import numpy as np

# Calculate distances of each point from the optimal line
distances = np.abs(y_pred_original - y_test_original)

# Find the indices of points with maximum distances
max_distance_indices = np.argsort(distances)[-30:]  # Adjust the number to show more or fewer names

# Get the corresponding player names
furthest_names = z.iloc[max_distance_indices]['name'].tolist()

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test_original, y_pred_original, s=20, color='blue', alpha=0.5, marker='o', label='Predicted Values')
plt.xlabel("True Value (EUR)")
plt.ylabel("Predicted Value (EUR)")
plt.title("True Value vs Predicted Value (XGBoost)")
plt.plot([min(y_test_original), max(y_test_original)], [min(y_test_original), max(y_test_original)], color='red', label='Perfect Prediction Line')
plt.text(min(y_test_original), max(y_pred_original), "Overestimation", fontsize=10, color='green', verticalalignment='top', horizontalalignment='left')
plt.text(max(y_test_original), min(y_pred_original), "Underestimation", fontsize=10, color='orange', verticalalignment='bottom', horizontalalignment='right')

# Annotate only the furthest points
for index in max_distance_indices:
    row = z.iloc[index]
    plt.annotate(row['name'], (row['market_value_in_eur'], row['Predicted_Value']), fontsize=6, color='black')

# Set formatter for x-axis and y-axis
def millions_formatter(x, pos):
    return f'{x / 1000000:.0f}M'

# Identify the 1% most valuable players
top_1_percent = z.nlargest(int(len(z) * 0.05), 'market_value_in_eur')
top_1_percent = top_1_percent.sort_values(by='market_value_in_eur')
lowest_value_player = top_1_percent.iloc[0]['market_value_in_eur']
# Add vertical line for the 1% most valuable players
print(lowest_value_player)
plt.axvline(x=lowest_value_player, color='gray', linestyle='--', label='5% Most Valuable Players', alpha=0.5)
    
formatter = FuncFormatter(millions_formatter)
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.grid(alpha=0.2, linestyle='--', zorder=1)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
XGBRegressor.plot_importance(model)